# Benchmark Postgres with Q3C extension

In [ ]:
# First make sure our services are running
! docker compose up -d

In [ ]:
import psycopg2

# Now drop the database if it exists and create a table for our
# alerts
sql_drop = "DROP DATABASE IF EXISTS boom;"
sql_create = "CREATE DATABASE boom;"

# Connect to the PostgreSQL database and execute the SQL commands
conn = psycopg2.connect(
    host="localhost", port=5432, user="postgres", password="postgres"
)
conn.autocommit = True  # Enable autocommit mode
cursor = conn.cursor()
cursor.execute(sql_drop)
cursor.execute(sql_create)
print("Database created successfully")

In [ ]:
# Check that q3c is installed
conn = psycopg2.connect(
    host="localhost",
    port=5432,
    user="postgres",
    password="postgres",
    database="boom",
)
# Get the q3c version
cursor = conn.cursor()
res = cursor.execute(
    """
        CREATE EXTENSION IF NOT EXISTS q3c;
        SELECT q3c_version()
    """
)
cursor.fetchall()

In [ ]:
import psycopg2

# Now drop the database if it exists and create a table for our
# alerts
sql = """
-- Enable the q3c extension for spatial indexing
CREATE EXTENSION IF NOT EXISTS q3c;

CREATE TABLE IF NOT EXISTS alert (
    id SERIAL PRIMARY KEY,
    survey_id INT NOT NULL,
    object_id VARCHAR(255) NOT NULL,
    cand_id VARCHAR(255) NOT NULL,
    candidate JSONB NOT NULL,
    ra DOUBLE PRECISION NOT NULL,
    dec DOUBLE PRECISION NOT NULL,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

-- Create a Q3C index on the ra and dec columns for faster querying
CREATE INDEX idx_objects_q3c ON alert (q3c_ang2ipix(ra, dec));
"""

# Connect to the PostgreSQL database and execute the SQL commands
conn = psycopg2.connect(
    host="localhost",
    port=5432,
    user="postgres",
    password="postgres",
    database="boom",
)

with conn.cursor() as cursor:
    cursor.execute(sql)
    conn.commit()
    print("Table created successfully.")

In [ ]:
# Insert all the ZTF alerts